<a href="https://colab.research.google.com/github/mjkimcs/portfolio/blob/main/SQL/Python_Oracle_GUI_%EC%8B%AC%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cx_Oracle

def Create_Table():
    dsn = cx_Oracle.makedsn("localhost", 1521, 'xe')
    db = cx_Oracle.connect('SCOTT', 'TIGER', dsn)
    cur = db.cursor()

    sql_cmd = "CREATE TABLE Test_mydb(id number, name varchar2(20))"
    cur.execute(sql_cmd)
    db.close()

def Insert_m():
    dsn = cx_Oracle.makedsn("localhost", 1521, 'xe')
    db = cx_Oracle.connect('SCOTT', 'TIGER', dsn)
    cur = db.cursor()

    cur.execute("INSERT INTO TEST_MYDB VALUES('1','PYTHON')")
    cur.execute("INSERT INTO TEST_MYDB VALUES('2','PYTHON2')")
    cur.execute("INSERT INTO TEST_MYDB VALUES('3','PYTHON3')")
    cur.execute("INSERT INTO TEST_MYDB VALUES('4','PYTHON4')")
    db.commit()
    db.close()

def Select_all():
    dsn = cx_Oracle.makedsn("localhost", 1521, 'xe')
    db = cx_Oracle.connect('SCOTT', 'TIGER', dsn)
    cur = db.cursor()

    cur.execute("select * from Test_mydb")
    result = cur.fetchall()
    for row in result:
        print(str(row[0]) + " " + row[1])
    cur.close()
    db.close()


if __name__ == '__main__':
    Create_Table()
    Insert_m()
    Select_all()

In [ ]:
import cx_Oracle

class MyClass:
    def __init__(self):
        self.dsn = cx_Oracle.makedsn("localhost", 1521, 'xe')
        self.db = cx_Oracle.connect('SCOTT', 'TIGER', self.dsn)
        self.cur = self.db.cursor()

    def Create_Table(self):
        sql_cmd = "CREATE TABLE Test_MYDB(id number, name varchar2(20))"
        self.cur.execute(sql_cmd)
        self.db.close()

    def insert_m(self):
        self.cur.execute("insert into test_mydb values('1','python1')")
        self.cur.execute("insert into test_mydb values('2','python2')")
        self.cur.execute("insert into test_mydb values('3','python3')")
        self.cur.execute("insert into test_mydb values('4','python4')")
        self.db.commit()
        self.db.close()

    def select_all(self):
        self.cur.execute("select * from Test_mydb")
        result = self.cur.fetchall()
        for row in result:
            print(str(row[0]) + " " + row[1]);
        self.cur.close()
        self.db.close()

    # def __del__(self):
    #     self.db.close()


if __name__ == '__main__':
    test = MyClass()
    # test.Create_Table()
    # test.insert_m()
    test.select_all()

In [ ]:
import cx_Oracle
from bs4 import BeautifulSoup
import urllib.request as req
import urllib.parse as par
import requests
import os


class DbClass:
    def __init__(self, search, rating_num, review_num):
        self.dsn = cx_Oracle.makedsn('localhost', 1521, 'xe')
        self.db = cx_Oracle.connect('SCOTT', 'TIGER', self.dsn)
        self.cur = self.db.cursor()

        self.search = search
        self.rating_num = rating_num
        self.review_num = review_num

    def __del__(self):
        self.db.close()

    def Create_Table(self):
        sql_cmd = "CREATE TABLE GUI_DB(PNAME VARCHAR2(1000), IMG_LINK VARCHAR2(1000), RATING NUMBER, REVIEW_NUM NUMBER, PRO_LINK VARCHAR2(1000))"
        self.cur.execute(sql_cmd)
        # self.db.close()

    def Coupang(self):
        encoded = par.quote(self.search)

        pg_num = 1
        row_num = 1
        while True:
            if pg_num == 5:
                break
            url = "https://www.coupang.com/np/search?q={}&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page={}&rocketAll=false&searchIndexingToken=1=4&backgroundColor=".format(
                encoded, pg_num)
            code = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
            soup = BeautifulSoup(code.text, "html.parser")

            all = soup.find_all("li", class_="search-product")
            name_list = []
            imlink_list = []
            rating_list = []
            review_list = []
            prlink_list = []
            for i in all:
                name = i.find("div", class_="name")
                rating = i.find("em", class_="rating")
                review = i.find("span", class_="rating-total-count")
                link = i.find("a")
                img = i.find("img", class_="search-product-wrap-img")
                if rating:
                    rating_f = rating.get_text(" ", strip=True)
                else:
                    continue
                if review:
                    review_f = review.get_text(" ", strip=True).replace("(", "").replace(")", "")
                else:
                    continue


                if float(rating_f) >= float(self.rating_num) and float(review_f) >= float(self.review_num):
                    name_f = name.get_text(" ",strip = True)
                    name_list.append(name_f)
                    print("[", pg_num, "페이지", "]")
                    print("상품명 :", row_num, name_f)
                    rating_list.append(rating_f)
                    print("평점 :", rating_f)
                    review_list.append(review_f)
                    print("리뷰수 :", review_f)
                    link_f = "https://www.coupang.com" + link["href"]
                    prlink_list.append(link_f)
                    print("링크 :", link_f)
                    print("===============================================================")
                    img_f = "https:" + img["src"]
                    if ".gif" in img_f:
                        img_f = "https:" + img["data-img-src"]
                    imlink_list.append(img_f)
                    row_num += 1
            pg_num += 1
            return [name_list, imlink_list, rating_list, review_list, prlink_list]


# class DbClass2(DbClass):
#     def __init__(self, search, rating_num, review_num):
#         DbClass.__init__(self, search, rating_num, review_num)
#         # super().__init__(search, rating_num, review_num)

    def Insert(self):
        for i, j, k, l, m in zip(self.Coupang()[0], self.Coupang()[1], self.Coupang()[2], self.Coupang()[3], self.Coupang()[4]):
            self.dsn = cx_Oracle.makedsn('localhost', 1521, 'xe')
            self.db = cx_Oracle.connect('SCOTT', 'TIGER', self.dsn)
            self.cur = self.db.cursor()

            my_dict = [(i, j, k, l, m)]
            query = "INSERT INTO GUI_DB(PNAME, IMG_LINK, RATING, REVIEW_NUM, PRO_LINK) VALUES(:1, :2, :3, :4, :5)"
            self.cur.executemany(query, my_dict)

            self.db.commit()
            # self.db.close()

    def Select_all(self):
        self.cur.execute("SELECT * FROM GUI_DB")
        result = self.cur.fetchall()
        for row in result:
            print(str(row[0]) + ", " + row[1] + ", " + str(row[2]) + ", " + str(row[3]) + ", " + row[4])
            print()
        # self.cur.close()
        # self.db.close()

    def Select_pro(self):
        res = self.cur.var(cx_Oracle.CURSOR)
        self.cur.callproc("GUI_PRO01", [res])
        res = res.getvalue()
        for i in res:
            print(i)
            # print(i[0]) # 제품명
        # self.cur.close()
        # self.db.close()


if __name__ == '__main__':
    test = DbClass('빔프로젝터', 4.5, 100)
    # test.Create_Table()
    test.Coupang()
    test.Insert()
    test.Select_all()
    test.Select_pro()
    del test

    # test2 = DbClass2('빔프로젝터', 4.5, 100)
    # test2.Insert()
    # test2.Select_all()

In [ ]:
# ORACLE
CREATE OR REPLACE PROCEDURE GUI_PRO01
(P_GUI OUT SYS_REFCURSOR)
AS
BEGIN
    OPEN P_GUI FOR
        SELECT PNAME, IMG_LINK
        FROM GUI_DB
        WHERE ROWNUM < 4;
END GUI_PRO01;


VAR RES REFCURSOR;
EXEC GUI_PRO01(:RES);
PRINT RES;

In [ ]:
# GUI
from PyQt5.QtWidgets import *
from PyQt5.QtCore import QObject, QThread, pyqtSignal
from PyQt5 import uic
import sys
import os
import urllib.request as req
from bs4 import BeautifulSoup
from PyQt5.QtGui import QPixmap
import urllib.parse as par
import requests

import cx_Oracle
from bs4 import BeautifulSoup
import urllib.request as req
import urllib.parse as par
import requests
import os


ui_file = "./recomm.ui"
class MainDialog(QDialog):
    dbtable_signal = pyqtSignal()
    crawl_insert_signal = pyqtSignal()
    show_signal = pyqtSignal()
    select_pro_signal = pyqtSignal()

    def __init__(self):
        QDialog.__init__(self, None)
        uic.loadUi(ui_file, self)

        self.id = ""
        self.rating = ""
        self.review = ""

        self.button_search.clicked.connect(self.SearchData)  # 1
        self.dbtable_signal.connect(self.Create_Table)  # 4
        self.crawl_insert_signal.connect(self.Coupang)
        self.show_signal.connect(self.DboutShow)
        self.select_pro_signal.connect(self.Select_pro)

        self.dsn = cx_Oracle.makedsn('localhost', 1521, 'xe')
        self.db = cx_Oracle.connect('SCOTT', 'TIGER', self.dsn)
        self.cur = self.db.cursor()

    def __del__(self):
        self.db.close()

    def SearchData(self):  # 2
        self.id = self.input_id.text()
        self.rating = self.input_rating.text()
        self.review = self.input_review.text()

        self.dbtable_signal.emit()  # 3

    def Create_Table(self):
        sql_cmd = "CREATE TABLE GUI_DB(PNAME VARCHAR2(1000), IMG_LINK VARCHAR2(1000), RATING NUMBER, REVIEW_NUM NUMBER, PRO_LINK VARCHAR2(1000))"
        self.cur.execute(sql_cmd)
        self.crawl_insert_signal.emit()

    def Coupang(self):
        encoded = par.quote(self.id)

        pg_num = 1
        row_num = 1
        while True:
            if pg_num == 5:
                break
            url = "https://www.coupang.com/np/search?q={}&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page={}&rocketAll=false&searchIndexingToken=1=4&backgroundColor=".format(
                encoded, pg_num)
            code = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
            soup = BeautifulSoup(code.text, "html.parser")

            all = soup.find_all("li", class_="search-product")
            name_list = []
            imlink_list = []
            rating_list = []
            review_list = []
            prlink_list = []
            for i in all:
                name = i.find("div", class_="name")
                rating = i.find("em", class_="rating")
                review = i.find("span", class_="rating-total-count")
                link = i.find("a")
                img = i.find("img", class_="search-product-wrap-img")
                if rating:
                    rating_f = rating.get_text(" ", strip=True)
                else:
                    continue
                if review:
                    review_f = review.get_text(" ", strip=True).replace("(", "").replace(")", "")
                else:
                    continue

                if float(rating_f) >= float(self.rating) and float(review_f) >= float(self.review):
                    name_f = name.get_text(" ", strip=True)
                    name_list.append(name_f)
                    print("[", pg_num, "페이지", "]")
                    print("상품명 :", row_num, name_f)
                    rating_list.append(rating_f)
                    print("평점 :", rating_f)
                    review_list.append(review_f)
                    print("리뷰수 :", review_f)
                    link_f = "https://www.coupang.com" + link["href"]
                    prlink_list.append(link_f)
                    print("링크 :", link_f)
                    print("===============================================================")
                    img_f = "https:" + img["src"]
                    if ".gif" in img_f:
                        img_f = "https:" + img["data-img-src"]
                    imlink_list.append(img_f)
                    row_num += 1
            pg_num += 1
            # return [name_list, imlink_list, rating_list, review_list, prlink_list]


            self.dsn = cx_Oracle.makedsn('localhost', 1521, 'xe')
            self.db = cx_Oracle.connect('SCOTT', 'TIGER', self.dsn)
            self.cur = self.db.cursor()

            my_dict = [(name_f, img_f, rating_f, review_f, link_f)]
            query = "INSERT INTO GUI_DB(PNAME, IMG_LINK, RATING, REVIEW_NUM, PRO_LINK) VALUES(:1, :2, :3, :4, :5)"
            self.cur.executemany(query, my_dict)

            self.db.commit()
        self.show_signal.emit()

    def DboutShow(self):
        res = self.cur.var(cx_Oracle.CURSOR)
        self.cur.callproc("GUI_PRO01", [res])
        res = res.getvalue()
        rank = 1
        for i in res:
            getattr(self, "txt{}".format(rank)).setText(
                "{}위 : {}".format(rank, i[0]))

            img_open = req.urlopen(i[1]).read()
            pixmap = QPixmap()
            pixmap.loadFromData(img_open)
            pixmap = pixmap.scaled(185, 260)
            getattr(self, "img{}".format(rank)).setPixmap(pixmap)
            getattr(self, "txt{}_2".format(rank)).append(f"구매하러 가기 : {i[2]}")
            rank += 1
        self.select_pro_signal.emit()

    def Select_pro(self):
        res = self.cur.var(cx_Oracle.CURSOR)
        self.cur.callproc("GUI_PRO01", [res])
        res = res.getvalue()
        rank = 1
        for i in res:
            print(f"<{rank}위>")
            print("제품명 : ", i[0])
            print("구매하기 : ", i[2])
            rank += 1


if __name__ == '__main__':
    QApplication.setStyle("fusion")
    app = QApplication(sys.argv)
    gtest = MainDialog()
    gtest.show()
    sys.exit(app.exec_())